In [ ]:
def predict(row,weights):
    activation=weights[0]
    for i in range(len(row)-1):
        activation+=weights[i+1]*row[i]
    return 1.0 if activation>=0.0 else 0.0

def train_weights(train,l_rate,n_epoch):
    weights=[0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        sum_error=0.0
        for row in train:
            prediction=predict(row,weights)
            error=row[-1]-prediction
            sum_error+=error**2
            weights[0]=weights[0]+l_rate*error
            for i in range(len(row)-1):
                weights[i+1]=weights[i+1]+l_rate*error*row[i]
        print('>epoch=%d, l_rate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
    return weights
            
dataset = [[2.7810836,2.550537003,0],
 [1.465489372,2.362125076,0],
 [3.396561688,4.400293529,0],
 [1.38807019,1.850220317,0],
 [3.06407232,3.005305973,0],
 [7.627531214,2.759262235,1],
 [5.332441248,2.088626775,1],
 [6.922596716,1.77106367,1],
 [8.675418651,-0.242068655,1],
 [7.673756466,3.508563011,1]]
l_rate = 0.1
n_epoch = 5
weights = train_weights(dataset, l_rate, n_epoch)
print(weights)

In [4]:
from random import seed
from random import random

#initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
    network=list()
    hidden_layer=[{'weights':[random() for i in range(n_inputs+1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer=[{'weights':[random() for i in range(n_hidden+1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

seed(1)
network=initialize_network(2, 1, 2)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


In [5]:
def activate(weights, inputs):
    activation=weights[-1]
    for i in range(len(weights)-1):
        activation+=weights[i]*inputs[i]
        return activation

In [6]:
def transfer(activation):
    return 1.0/(1.0+exp(-activation))

In [13]:
#Forward proprogate 
def forward_propogate(network, row):
    inputs=row
    for layer in network:
        new_inputs=[]
        for neuron in layer:
            activation=activate(neuron['weights'], inputs)
            neuron['output']=transfer(activation)
            new_inputs.append(neuron['output'])
        inputs=new_inputs
    return inputs

In [15]:
from math import exp
network = [[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
 [{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]]
row=[1,0,None]
output=forward_propogate(network, row)
print(output)

[0.6629970129852887, 0.7253160725279748]


In [25]:
#calculate the derivative
def transfer_derivative(output):
    return output*(1.0-output)

In [26]:
error=(weight_k*error_j)*transfer_derivative(output)

NameError: name 'weight_k' is not defined

In [50]:
#backpropagate
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer=network[i]
        errors=list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error=0.0
                for neuron in network[i+1]:
                    error+=(neuron['weights'][j]*neuron['delta'])
                    errors.append(error)
        else:
            for j in range(len(layer)):
                neuron=layer[j]
                errors.append(neuron['output']-expected[j])
        for j in range(len(layer)):
            neuron=layer[j]
            neuron['delta']=errors[j]*transfer_derivative(neuron['output'])

In [51]:
network = [[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
 [{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]]
expected = [0, 1]
backward_propagate_error(network, expected)
for layer in network:
    print(layer)

[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'delta': 0.007668854370284511}]
[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': 0.14619064683582808}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': -0.0771723774346327}]


In [53]:
#update network
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs=row[:-1]
        if i != 0:
            inputs=[nueron['output'] for nueron in network[i-1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j]-=l_rate*neuron['delta']*inputs[j]
            neuron['weights'][-1]-=l_rate*neuron['delta']
        

In [54]:
#train a network
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error=0
        for row in train:
            outputs=forward_propogate(network, row)
            expected=[0 for i in range(n_outputs)]
            expected[row[-1]]=1
            sum_error+=sum([(expected[i]-output[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' %(epoch, l_rate, sum_error))
            # Test training backprop algorithm
seed(1)
dataset = [[2.7810836,2.550537003,0],
 [1.465489372,2.362125076,0],
 [3.396561688,4.400293529,0],
 [1.38807019,1.850220317,0],
 [3.06407232,3.005305973,0],
 [7.627531214,2.759262235,1],
 [5.332441248,2.088626775,1],
 [6.922596716,1.77106367,1],
 [8.675418651,-0.242068655,1],
 [7.673756466,3.508563011,1]]
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
 print(layer)
            

>epoch=0, lrate=0.500, error=5.773
>epoch=1, lrate=0.500, error=5.773
>epoch=2, lrate=0.500, error=5.773
>epoch=3, lrate=0.500, error=5.773
>epoch=4, lrate=0.500, error=5.773
>epoch=5, lrate=0.500, error=5.773
>epoch=6, lrate=0.500, error=5.773
>epoch=7, lrate=0.500, error=5.773
>epoch=8, lrate=0.500, error=5.773
>epoch=9, lrate=0.500, error=5.773
>epoch=10, lrate=0.500, error=5.773
>epoch=11, lrate=0.500, error=5.773
>epoch=12, lrate=0.500, error=5.773
>epoch=13, lrate=0.500, error=5.773
>epoch=14, lrate=0.500, error=5.773
>epoch=15, lrate=0.500, error=5.773
>epoch=16, lrate=0.500, error=5.773
>epoch=17, lrate=0.500, error=5.773
>epoch=18, lrate=0.500, error=5.773
>epoch=19, lrate=0.500, error=5.773
[{'weights': [-0.7095684677535904, 3.5039052971069213, 1.5242469432734262], 'output': 0.021590799567244284, 'delta': 0.0035783656017940716}, {'weights': [-0.8936533805118394, 5.034248982818817, 1.8213179217762523], 'output': 0.0069011455949210265, 'delta': 0.0022448350803103327}]
[{'weight

In [4]:
from math import exp
from random import seed
from random import random

# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation

# Transfer neuron activation
def transfer(activation):
	return 1.0 / (1.0 + exp(-activation))

# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(neuron['output'] - expected[j])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] -= l_rate * neuron['delta']

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

# Test training backprop algorithm
seed(1)
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
	print(layer)

TabError: inconsistent use of tabs and spaces in indentation (3284835454.py, line 8)

In [2]:
from random import random,seed
seed(1)

## Network Init
def initialize_network(n_inputs, n_hidden, n_outputs):
 network = list()
 hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
 network.append(hidden_layer)
 output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
 network.append(output_layer)
 return network

## Activation Function
def activate(weights, input):
  activation = weights[-1]
  for i in range(len(weights)-1):
    activation += weights[i] * input[i]
  return activation

## Neuron Transfer
from math import exp
def transfer(activation):
  return 1.0 / (1.0 + exp(-activation))

## Forward Propogation
def forward_propagate(network, row):
 inputs = row
 for layer in network:
  new_inputs = []
  for neuron in layer:
    activation = activate(neuron['weights'], inputs)
    neuron['output'] = transfer(activation)
    new_inputs.append(neuron['output'])
  inputs = new_inputs
 return inputs

## Neuron Transfer
def transfer_derivative(output):
 return output * (1.0 - output)

## Backward Propogation
def backward_propagate_error(network, expected):
 for i in reversed(range(len(network))):
  layer = network[i]
  errors = list()
  if i != len(network)-1:
    for j in range(len(layer)):
      error = 0.0
      for neuron in network[i + 1]:
        error += (neuron['weights'][j] * neuron['delta'])
        errors.append(error)
  else:
    for j in range(len(layer)):
      neuron = layer[j]
      errors.append(neuron['output'] - expected[j])
  for j in range(len(layer)):
    neuron = layer[j]
    neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

## Weights Update
def update_weights(network, row, l_rate):
 for i in range(len(network)):
  inputs = row[:-1]
  if i != 0:
    inputs = [neuron['output'] for neuron in network[i - 1]]
    for neuron in network[i]:
      for j in range(len(inputs)):
        neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
      neuron['weights'][-1] -= l_rate * neuron['delta']

## Train Network
def train_network(network, train, l_rate, n_epoch, n_outputs):
 for epoch in range(n_epoch):
  sum_error = 0
  for row in train:
      outputs = forward_propagate(network, row)
      expected = [0 for i in range(n_outputs)]
      expected[row[-1]] = 1
      sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
      backward_propagate_error(network, expected)
      update_weights(network, row, l_rate)
  print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))


dataset =  [[1,1,0],
            [1,0,1],
            [0,1,1],
            [0,0,0]]
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
print(f"n_inputs: {n_inputs}, n_outputs: {n_outputs}")
network = initialize_network(n_inputs, 1, n_outputs)
print(f"Network: {network}")
train_network(network, dataset, 0.01, 10, n_outputs)
for layer in network:
 print(layer)

n_inputs: 2, n_outputs: 2
Network: [[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}], [{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]]
>epoch=0, lrate=0.010, error=2.324
>epoch=1, lrate=0.010, error=2.322
>epoch=2, lrate=0.010, error=2.320
>epoch=3, lrate=0.010, error=2.319
>epoch=4, lrate=0.010, error=2.317
>epoch=5, lrate=0.010, error=2.315
>epoch=6, lrate=0.010, error=2.313
>epoch=7, lrate=0.010, error=2.312
>epoch=8, lrate=0.010, error=2.310
>epoch=9, lrate=0.010, error=2.308
[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'output': 0.6821726828457035, 'delta': -0.004090947989998892}]
[{'weights': [0.24369089678413863, 0.4808384243937026], 'output': 0.6560951098437028, 'delta': -0.07759674489431784}, {'weights': [0.4357092308213479, 0.633551249750052], 'output': 0.7176654973580177, 'delta': 0.14541462554016926}]
